#### Disclaimer: all of the code below was written and tested from me (Salvatore Latino), and Andrea Panuccio, a collegue

In [ ]:
from random import random
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue, SimpleQueue, LifoQueue
from math import ceil

import numpy as np

In [ ]:
PROBLEM_SIZE = 15
NUM_SETS = 75
SETS = tuple(np.array([random() < .2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS)) #Initialize the set
State = namedtuple('State', ['taken', 'not_taken'])

In [ ]:
def goal_check(state, source):
    return np.all(reduce(np.logical_or, [source[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))) #Function to check if we found a solution

In [ ]:
assert goal_check(State(set(range(NUM_SETS)), set()), SETS), "Probelm not solvable" #We are sure the problem is solvable

In [ ]:
def covered(state, source):
    return reduce(np.logical_or,[source[i] for i in state.taken],np.array([False for _ in range(PROBLEM_SIZE)]),) #Tiles we have covered

def distance(state,source):
    return PROBLEM_SIZE - sum(
        reduce(np.logical_or,[source[i] for i in state.taken],np.array([False for _ in range(PROBLEM_SIZE)]))) #How many tiles to cover yet

### In previous solution me and Andrea tried to remove all the duplicates in SETS by adding them to a set, in order to speed up the algorithms.
### To do that we made a custom hashable class (My Tile) 

In [ ]:
def compute_hash(tile):
    n = 0
    for i in range(PROBLEM_SIZE):
        n = 2 * n + tile[i]
    return int(n)

In [ ]:
class MyTile:
    def __init__(self, data):
        self.data = data

    def __eq__(self, other):
        if isinstance(other, MyTile):
            return (self.data == other.data).all()
        return False
    
    def __ne__(self, other):
        if isinstance(other, MyTile):
            return (self.data != other.data).all()
        return True
    
    def __hash__(self):
        return compute_hash(self.data)

In [ ]:
#tiles_set = set((map(lambda t : MyTile(t), SETS)))

### Then we realized we could obtain the same result (discard the duplicates) just by this line of code

In [ ]:
tiles_no_duplicates = list(set(map( lambda x: tuple(x) , SETS))) #DISCARD DUPLICATES IN SETS 

### As another optimization we thought about special set. A special set for us is a set that covers a tile that isn't covered by any other set and therefore must be present in solution. Of course this would really effective when the prob for a set to cover a specific tile is very low.

In [ ]:
initial_taken = set()
initial_not_taken = set(range(len(tiles_no_duplicates)))

for i in range(PROBLEM_SIZE):
    counter = 0
    pos = -1
    for j in range(len(tiles_no_duplicates)):
        if tiles_no_duplicates[j][i]:
            counter += 1
            if counter > 1:
                break
            pos = j
    if counter == 1:
        initial_taken.add(pos)
        initial_not_taken.remove(pos)

## Breadth first W/o optimization

In [ ]:
"""
frontier = SimpleQueue()
frontier.put(State(set(), set(range(NUM_SETS)))) #I take every available set

counter = 0
current_state = frontier.get()
while not np.all(reduce(np.logical_or, [SETS[i] for i in current_state.taken], np.array ([False for _ in range(PROBLEM_SIZE)]))):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put(new_state)
    current_state = frontier.get()

print(f"Solved in {counter:,} steps")

cnt_bf_no_opt = counter
sol_bf_no_opt = current_state.taken
"""

## Breadth first W/ optimization

In [ ]:
"""
frontier = SimpleQueue()
frontier.put(State(initial_taken, initial_not_taken)) #Discard duplicates

counter = 0
current_state = frontier.get()
while not np.all(reduce(np.logical_or, [tiles_no_duplicates[i] for i in current_state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put(new_state)
    current_state = frontier.get()

print(f"Solved in {counter:,} steps")

cnt_bf_opt = counter
sol_bf_opt = current_state.taken
"""

## Greedy

### As a greedy problem it's not optimal, however in some cases we saw that it's capable of give an optimal solution in less step and time than A*  

In [ ]:
frontier = PriorityQueue()
frontier.put(State(set(), set(range(NUM_SETS))))

counter = 0
current_state = frontier.get()
while not np.all(reduce(np.logical_or, [SETS[i] for i in current_state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        frontier.put(((len(new_state.taken)**2)*distance(new_state,SETS),new_state)) #Starting from a wrong heuristic for A* we found this function for our greedy algorithm
    current_state = frontier.get()[1]

print(f"Solved in {counter:,} steps")

cnt_gr_no_opt = counter
sol_gr_no_opt = current_state.taken

## A* 

In [ ]:
def sol_or_not(state):
    if np.all(reduce(np.logical_or, [tiles_no_duplicates[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))):
        return 0
    return 1

## A* W/ optimization

In [ ]:
initial_taken = set()
initial_not_taken = set(range(len(tiles_no_duplicates)))

frontier = PriorityQueue()
frontier.put(State(initial_taken, initial_not_taken))

counter = 0
current_state = frontier.get()
while not np.all(reduce(np.logical_or, [tiles_no_duplicates[i] for i in current_state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        #frontier.put((len(current_state.taken)+distance(current_state,tiles_no_duplicates),new_state)) Me and Andrea incorrectly used a pessimistic heuristic at first, based on how many sets we have in solution and how many tiles were missing
        frontier.put((len(new_state.taken)+sol_or_not(new_state),new_state)) # Then we used a very simple optimistic heuristic
    current_state = frontier.get()[1]

print(f"Solved in {counter:,} steps")

cnt_as_opt = counter
sol_as_opt = current_state.taken

## A* with mem of already explored solution

### The order of the sets is not important in this problem. In addition to that, if two selection of sets lead to the same tiles covered is not useful exploring both

In [ ]:
initial_taken = set()
initial_not_taken = set(range(len(tiles_no_duplicates)))

frontier = PriorityQueue()
frontier.put(State(initial_taken, initial_not_taken))

explored = set()

counter = 0
current_state = frontier.get()
while not np.all(reduce(np.logical_or, [tiles_no_duplicates[i] for i in current_state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
        old_len = len(explored)
        x = covered(new_state, tiles_no_duplicates).tolist()
        x.append(len(new_state.taken)) #We must ensure that a selection of set that covers the same tiles as a previous one but in less number of tiles will be expanded
        explored.add(tuple(x))
        new_len = len(explored)
        if (new_len > old_len):
            frontier.put((len(new_state.taken)+sol_or_not(new_state),new_state))
    current_state = frontier.get()[1]

print(f"Solved in {counter:,} steps")

cnt_as_opt_exp = counter
sol_as_opt_exp = current_state.taken

## A* W/ optimization+

### Here we take the heuristic given from the professor and make just a little adjustment taking the candidates only from the tiles not in solution. We saw that even if it solves the problem in less step than the other version we developed most of the times it's slower than the other in seconds

In [ ]:
def exp_cost(state):
    already_covered = covered(state, tiles_no_duplicates)
    if np.all(already_covered):
        return 0
    missing_size = PROBLEM_SIZE - sum(already_covered)
    candidates = sorted((sum(np.logical_and(tiles_no_duplicates[s], np.logical_not(already_covered))) for s in state.not_taken), reverse=True)
    taken = 1
    while sum(candidates[:taken]) < missing_size:
        taken += 1
    return taken

In [ ]:
frontier = PriorityQueue()
state = State(set(), set(range(len(tiles_no_duplicates))))
frontier.put((len(state.taken) + exp_cost(state), state))

counter = 0
_, current_state = frontier.get()
while not np.all(reduce(np.logical_or, [tiles_no_duplicates[i] for i in current_state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))):
    counter += 1
    for action in current_state[1]:
        new_state = State(current_state.taken ^ {action},current_state.not_taken ^ {action},)
        frontier.put((len(new_state.taken) + exp_cost(new_state), new_state))
    _, current_state = frontier.get()

cnt_as_opt_plus = counter
sol_as_opt_plus = current_state.taken

## RESULTS

In [ ]:
print(f"{len(SETS)-len(tiles_no_duplicates)} duplicates sets were discarded")
print(f"{len(initial_taken)} sets were added to the solution")
print()

#print(f"Breadth First W/o optimization:  Sol: {sol_bf_no_opt}   Solved in {cnt_bf_no_opt} steps.   ")
#print(f"Breadth First W/ optimization:   Sol: {sol_bf_opt}   Solved in {cnt_bf_opt}  steps. ")
#print(f"A* W/o optimization:             Sol: {sol_as_no_opt}   Solved in {cnt_as_no_opt} steps.   ")
print(f"Greedy W/o optimization:          Sol: {sol_gr_no_opt}   Solved in {cnt_gr_no_opt} steps.  ")
print(f"A* W/ optimization:               Sol: {sol_as_opt}   Solved in {cnt_as_opt} steps.  ")
print(f"A* W/ optimization EXP:           Sol: {sol_as_opt_exp}   Solved in {cnt_as_opt_exp} steps.  ")
print(f"A* W/ optimization+:              Sol: {sol_as_opt_plus}   Solved in {cnt_as_opt_plus} steps.  ")